# LSTM IMDB Movie Review Tutorial
Josiah Olson

In [1]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, BatchNormalization, GRU
from keras.datasets import imdb

import os
from keras.preprocessing.text import Tokenizer

Using Theano backend.


In [2]:
max_features = 10000
max_len = 200  # cut texts after this number of words (among top max_features most common words)

In [3]:
# get dataset and unzip: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

X_train = []
y_train = []

path = 'C:/2_MyThesis/ThucNghiem/Data/VS_4D/out/train/pos/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([1 for _ in range(854)])

path = 'C:/2_MyThesis/ThucNghiem/Data/VS_4D/out/train/neg/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([0 for _ in range(238)])

In [4]:
X_test = []
y_test = []

path = 'C:/2_MyThesis/ThucNghiem/Data/VS_4D/out/test/pos/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([1 for _ in range(375)])

path = 'C:/2_MyThesis/ThucNghiem/Data/VS_4D/out/test/neg/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([0 for _ in range(88)])

In [5]:
#tokenize works to list of integers where each integer is a key to a word
imdbTokenizer = Tokenizer(nb_words=max_features)

imdbTokenizer.fit_on_texts(X_train)

In [6]:
#print top 20 words 
#note zero is reserved for non frequent words
for word, value in imdbTokenizer.word_index.items():
    if value < 20:
        print(value, word)

2 trong
13 giá
12 cho
8 có
1 và
18 đã
15 đầu
4 đồng
14 công
7 các
17 được
3 của
11 tăng
5 năm
10 tỷ
16 doanh
19 cổ
6 với
9 là


In [7]:
#create int to word dictionary
intToWord = {}
for word, value in imdbTokenizer.word_index.items():
    intToWord[value] = word

#add a symbol for null placeholder
intToWord[0] = "!!!NA!!!"
    
print(intToWord[1])
print(intToWord[2])
print(intToWord[32])

và
trong
sản


In [8]:
#convert word strings to integer sequence lists
print(X_train[0])
print(imdbTokenizer.texts_to_sequences(X_train[:1]))
for value in imdbTokenizer.texts_to_sequences(X_train[:1])[0]:
    print(intToWord[value])
    
X_train = imdbTokenizer.texts_to_sequences(X_train)
X_test = imdbTokenizer.texts_to_sequences(X_test)

 trong tổng_số mã cổ_phiếu giảm_giá trong năm mã giảm mạnh nhất với bao_gồm svn gtt hla ptk dc avf asa hly hsi và th nhóm ngành xây_dựng và bất_động_sản ngành xây_dựng ngày niêm_yết vốn điều_lệ đồng giá đã điều_chỉnh đầu năm nbsp đồng cpgiá tại ngày phiên cuối năm đồng cp tỷ_lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành vận_tải và kho_bãi ngành vận_chuyển khách đường_bộ hệ_thống trạm dừng ngày niêm_yết vốn điều_lệ đồnggiá đã điều_chỉnh đầu năm đồng cpgiá tại ngày phiên cuối năm đồng cptỷ lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành sản_xuất ngành kim_loại và các sản_phẩm từ khoáng phi kim_loại ngày niêm_yết vốn điều_lệ đồng giá đã điều_chỉnh đầu năm đồng cpgiá tại ngày phiên cuối năm đồng cp tỷ_lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành khai_khoáng ngành khai_khoáng khác ngày niêm_yết vốn điều_lệ đồnggiá đã điều_chỉnh đầu năm đồng cpgiá tại ngày phiên cuối năm đồng cptỷ lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành xây_dựng và bất_động_sả

In [9]:
# Censor the data by having a max review length (in number of words)

#use this function to load data from keras pickle instead of munging as shown above
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                      test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

1092 train sequences
463 test sequences
Pad sequences (samples x time)
X_train shape: (1092L, 200L)
X_test shape: (463L, 200L)


In [10]:
#example of a sentence sequence, note that lower integers are words that occur more commonly
print("x:", X_train[0]) #per observation vector of 20000 words
print("y:", y_train[0]) #positive or negative review encoding

x: [ 388  393   55   69  171    4   13   18   69  332   15    5    4 1829   35
  114  126  236    5    4  274   10  171   25  285  119   74  112  208  445
  289 2000  155   32   76  155  237  453  904  238  114  388  393   55   69
  171 3091   18   69  332   15    5    4 1829   35  114  126  236    5    4
  274   10  171   25  285  119   74  112  208  445  289 2000  155   32   76
  155  641  504    1    7   32  238   26  920  916  641  504  114  388  393
   55   69  171 3091   18   69  332   15    5    4 1829   35  114  126  236
    5    4  274   10  171   25  285  119   74  112  208  445  289 2000  155
   32   76  155  237  453  904  238  114  388  393   55   69  171    4   13
   18   69  332   15    5    4 1829   35  114  126  236    5    4  274   10
  171   25  285  119   74  112  208  445  289 2000  155  202  331  155   88
 1252  114  388  393   55   69  171 3091   18   69  332   15    5   80    4
 1829   35  114  126  236    5    4  274   10  171   25  285  119   74  112
  208  44

In [11]:
# double check that word sequences behave/final dimensions are as expected
print("y distribution:", np.unique(y_train, return_counts=True))
print("max x word:", np.max(X_train), "; min x word", np.min(X_train))
print("y distribution test:", np.unique(y_test, return_counts=True))
print("max x word test:", np.max(X_test), "; min x word", np.min(X_test))

y distribution: (array([0, 1]), array([238, 854], dtype=int64))
max x word: 6848 ; min x word 0
y distribution test: (array([0, 1]), array([ 88, 375], dtype=int64))
max x word test: 6803 ; min x word 0


In [12]:
print("most and least popular words: ")
print(np.unique(X_train, return_counts=True))
# as expected zero is the highly used word for words not in index

most and least popular words: 
(array([   0,    1,    2, ..., 6843, 6846, 6848]), array([6689, 2719, 2475, ...,    1,    1,    1], dtype=int64))


In [13]:
#set model hyper parameters
epochs = 5
embedding_neurons = 128
lstm_neurons = 64
batch_size = 32

Next let's import pre-trained word vectors from google and use them to initialize our embedding to see if this improves the neural net's performance.

In [14]:
from gensim.models import Word2Vec

#get pre trained word2vec from google:
#https://doc-0k-4g-docs.googleusercontent.com/docs/securesc/gnqvgap6hjncpd3b10i2tv865io48jas/hmjtdgee48c14e1parufukrpkb8urra5/1463018400000/06848720943842814915/09676831593570546402/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&nonce=4l49745nmtine&user=09676831593570546402&hash=i2qe9mshan4mesl112ct9bu1tj9kr1hq

googlew2v = Word2Vec.load_word2vec_format('C:/1_Research/Create_data/aclImdb/word2vector/GoogleNews-vectors-negative300.bin', binary=True)

C:\Anaconda2\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda2\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [15]:
# get word vectors for words in my index
googleVecs = []
for value in range(max_features):
    try:
        googleVecs.append(googlew2v[intToWord[value]])
    except:
        googleVecs.append(np.random.uniform(size=300))

googleVecs = np.array(googleVecs)

print(googleVecs)
print(googleVecs.shape)

[[ 0.26202468  0.15868397  0.27812652 ...,  0.10937801  0.75767728
   0.4428527 ]
 [ 0.96970407  0.12253514  0.35782362 ...,  0.97491444  0.6667405
   0.40817497]
 [-0.20117188  0.140625    0.00427246 ...,  0.02307129 -0.06298828
   0.32226562]
 ..., 
 [ 0.74221873  0.22051525  0.84255914 ...,  0.13028034  0.15021938
   0.21722988]
 [ 0.77492949  0.61239571  0.28688942 ...,  0.14056612  0.58534851
   0.62429266]
 [ 0.3628167   0.45290652  0.74855452 ...,  0.33012347  0.7255602
   0.0751601 ]]
(10000L, 300L)


C:\Anaconda2\lib\site-packages\gensim\models\word2vec.py:1578: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return self.syn0[self.vocab[words].index]


In [16]:
# Bi-directional google

# this example tests if using pretrained embeddings will improve performance 
# relative to starting with random embeddings

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, 300, input_length=max_len, weights=[googleVecs])(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4)(bnorm)
# apply backwards LSTM
backwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4, go_backwards=True)(bnorm)

# concatenate the outputs of the 2 LSTMs
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.5)(merged)
output = Dense(1, activation='sigmoid')(after_dp)

model_bidir_google = Model(input=sequence, output=output)
# review model structure
print(model_bidir_google.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 300)      3000000     input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 200, 300)      600         embedding_1[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 64)            70080       batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [17]:
# Bi-directional google
from keras.callbacks import ModelCheckpoint
model_bidir_google.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()


history_bidir_google = model_bidir_google.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)


Train...
Train on 1092 samples, validate on 463 samples
Epoch 1/5
28s - loss: 0.6634 - acc: 0.6786 - val_loss: 0.5488 - val_acc: 0.8099
Epoch 2/5
28s - loss: 0.5771 - acc: 0.7454 - val_loss: 0.5299 - val_acc: 0.8121
Epoch 3/5
30s - loss: 0.5393 - acc: 0.7701 - val_loss: 0.5282 - val_acc: 0.8099
Epoch 4/5
29s - loss: 0.5270 - acc: 0.7683 - val_loss: 0.5327 - val_acc: 0.8035
Epoch 5/5
29s - loss: 0.5035 - acc: 0.7729 - val_loss: 0.5403 - val_acc: 0.8035
avg sec per epoch: 33.7541999817
